# Bricklayer Robot

In [1]:
# Extensions: magic to avoid re-loading modified .py modules all the time
%load_ext autoreload
%autoreload 2

# Bookkeeping
import sys
import shutil
import os as os
import matplotlib.pyplot as plt

# Custom project classes and functions
from iiwa import *
from Trajectory import *
from TrajectoryBuilder import *
from Visualizer import *
from BrickPlanner import *

# Some paths
kProjectDir = os.getcwd()
print("Working in directory ", kProjectDir)

# Hack: move our model packages to the manipulation/ folder to allow Drake to find them
shutil.copyfile(kProjectDir + "/models/brick.dmd.yaml", kProjectDir + "/../manipulation/manipulation/models/brick.dmd.yaml")
shutil.copyfile(kProjectDir + "/models/real_brick.sdf", kProjectDir + "/../manipulation/manipulation/models/real_brick.sdf")
shutil.copyfile(kProjectDir + "/models/ground_model.sdf", kProjectDir + "/../manipulation/manipulation/models/ground_model.sdf")

# Start the visualizer.
meshcat = StartMeshcat()

/home/nikita/env/lib/python3.10/site-packages/mpld3/mplexporter/renderers/base.py:4: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
INFO:drake:Meshcat listening for connections at http://localhost:7000


Working in directory  /home/nikita/MIT_6_4212/project


In [2]:
# System configs
# Brick configuration
kNumberOfBricks =  100 # @param
# Brick dimensions, must be consistent with the geometry in models/real_brick.sdf
kBrickGeom = np.array([0.075, 0.1, 0.05]) # @param

# Brick warehouse config
kWhLocation = np.array([2.5, 2.5, 0.15]) # location of the warehouse (corner), # @param
kWhSize = np.array([5, 5, 10]) # size of the warehouse grid in cells, # @param
if (kWhSize.prod() < kNumberOfBricks):
    assert False, "The warehouse is too small to fit all bricks"

# Brick source
X_WBrickSource = RigidTransform(np.array([0.5, 0.5, kBrickGeom[2]/2])) # @param

In [3]:
# Instantiate the robot (TODO: remove it from here)
ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks, kBrickGeom)

In [26]:
# Generate bricks
floorplan = """
|
|
|
"""

floorplan_0 = """
|- - - - - |
|          |
|          |
|          |
|- - - - - |
"""

floorplan_1 = """
|- - - - - |
|          |
|- - -     |
      |    |
      |    |
|- - -     |
|          |
|- - - - - |
"""

kHeight = 4 # @param
kGap = 0.005 # @param
planer = Planner(floorplan_0, kBrickGeom, kGap, kHeight)
#planer.shift(0.3, -0.4)
planer.shift(-0.3, 0.4)
total_bricks_needed = planer.get_total_bricks_n()
if (total_bricks_needed > kNumberOfBricks):
    assert False, "There is not enough bricks to cover the building"
print("Bricks planned, total bricked needed: ", total_bricks_needed)

# Visualize floorplan
vis = Visualizer(meshcat, kBrickGeom)
vis.clear_vis()
vis.visualize_plan(np.array(planer.get_brick_poses()))

Parsing floorplan:  
|- - - - - |
|          |
|          |
|          |
|- - - - - |

Bricks planned, total bricked needed:  80


In [27]:
# Build trajectories
# Build initial trajectory and get q0 in the calibration point
trj_builder = TrajectoryBuilder(ilyich, X_WBrickSource)
trj_builder.gen_initial_traj()
err = trj_builder.solve_IK(np.array([ 0., 0.6, 0., 0.0, 0.0, 1, 0.0, 0., 0.]))
assert err == 0, "Failed to solve IK for the initial trajectory"

q0 = trj_builder.get_trajectories().get_traj()[-1][7]
print("q0 in calibration point is ", q0)

covered_bricks = []
failed_bricks = []
for i in range(0, total_bricks_needed):
    b = planer.get_brick_poses()[i]
    trj_builder_for_brick = TrajectoryBuilder(ilyich, X_WBrickSource)
    R = RotationMatrix()#RotationMatrix.MakeYRotation(-np.pi/2)# if b[3] == 0 else RotationMatrix()
    X_WBrickTarget = RigidTransform(R, b[:3])

    # Generate trajectory for brick
    trj_builder_for_brick.gen_grab_brick_traj(i)
    trj_builder_for_brick.gen_move_to_place_traj(X_WBrickTarget, i)
    trj_builder_for_brick.gen_place_brick_traj(X_WBrickTarget, b[3], i)
    trj_builder_for_brick.gen_return_to_source_traj(X_WBrickTarget, i)
    
    # Solve IK for the trajectories
    if not trj_builder_for_brick.solve_IK(q0) == 0:
        failed_bricks.append(i)
    else:
        # Append trajectories
        covered_bricks.append(i)
        trj_builder.merge(trj_builder_for_brick)

# Print stat
print("Covered bricks: ", covered_bricks)
print("Failed bricks:", failed_bricks)

# Speed-up a bit
trj_builder.get_trajectories().slow_down(0.5)

#trj_builder.get_trajectories().dump_trajectories([True, True, False, True, False, False])

# Make trajectories for iiwa with IK
# Form iiwa trajectories with IK
traj, finger_traj = trj_builder.get_trajectories().form_iiwa_traj_q()

q0 in calibration point is  [-2.01456465 -1.18269787  2.05091819 -0.82894531  1.04389826  1.6124684
  0.99042727  0.          0.        ]
Failed to solve IK for trajectory point:  place brick #3 approach (0.0)
Failed to solve IK for trajectory point:  place brick #3 approach (0.5)
Failed to solve IK for trajectory point:  place brick #3 approach (1.0)
Failed to solve IK for trajectory point:  place brick #3 approach (1.5)
Failed to solve IK for trajectory point:  place brick #3 approach (2.0)
Failed to solve IK for trajectory point:  place brick #3 approach (2.5)
Failed to solve IK for trajectory point:  place brick #3 approach (3.0)
Failed to solve IK for trajectory point:  place brick #3 approach (3.5)
Failed to solve IK for trajectory point:  place brick #3 finger open 1
Failed to solve IK for trajectory point:  place brick #3 finger open 2
Failed to solve IK for trajectory point:  place brick #3 withdraw (0.0)
Failed to solve IK for trajectory point:  place brick #3 withdraw (0.5)


Failed to solve IK for trajectory point:  move #8 time: 1.1
Failed to solve IK for trajectory point:  move #8 time: 1.2000000000000002
Failed to solve IK for trajectory point:  move #8 time: 1.3
Failed to solve IK for trajectory point:  move #8 time: 1.4000000000000001
Failed to solve IK for trajectory point:  move #8 time: 1.5
Failed to solve IK for trajectory point:  move #8 time: 1.6
Failed to solve IK for trajectory point:  move #8 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #8 time: 1.8
Failed to solve IK for trajectory point:  move #8 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #8 turn (0.0)
Failed to solve IK for trajectory point:  place brick #8 turn (0.5)
Failed to solve IK for trajectory point:  place brick #8 turn (1.0)
Failed to solve IK for trajectory point:  place brick #8 turn (1.5)
Failed to solve IK for trajectory point:  place brick #8 turn (2.0)
Failed to solve IK for trajectory point:  place brick #8 app

Failed to solve IK for trajectory point:  move #17 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #17 time: 1.8
Failed to solve IK for trajectory point:  move #17 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #17 approach (0.0)
Failed to solve IK for trajectory point:  place brick #17 approach (0.5)
Failed to solve IK for trajectory point:  place brick #17 approach (1.0)
Failed to solve IK for trajectory point:  place brick #17 approach (1.5)
Failed to solve IK for trajectory point:  place brick #17 approach (2.0)
Failed to solve IK for trajectory point:  place brick #17 approach (2.5)
Failed to solve IK for trajectory point:  place brick #17 approach (3.0)
Failed to solve IK for trajectory point:  place brick #17 approach (3.5)
Failed to solve IK for trajectory point:  place brick #17 finger open 1
Failed to solve IK for trajectory point:  place brick #17 finger open 2
Failed to solve IK for trajectory point:  place brick #17 w

Failed to solve IK for trajectory point:  move #25 time: 1.4000000000000001
Failed to solve IK for trajectory point:  move #25 time: 1.5
Failed to solve IK for trajectory point:  move #25 time: 1.6
Failed to solve IK for trajectory point:  move #25 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #25 time: 1.8
Failed to solve IK for trajectory point:  move #25 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #25 approach (0.0)
Failed to solve IK for trajectory point:  place brick #25 approach (0.5)
Failed to solve IK for trajectory point:  place brick #25 approach (1.0)
Failed to solve IK for trajectory point:  place brick #25 approach (1.5)
Failed to solve IK for trajectory point:  place brick #25 approach (2.0)
Failed to solve IK for trajectory point:  place brick #25 approach (2.5)
Failed to solve IK for trajectory point:  place brick #25 approach (3.0)
Failed to solve IK for trajectory point:  place brick #25 approach (3.5)
Faile

Failed to solve IK for trajectory point:  move #30 time: 1.0
Failed to solve IK for trajectory point:  move #30 time: 1.1
Failed to solve IK for trajectory point:  move #30 time: 1.2000000000000002
Failed to solve IK for trajectory point:  move #30 time: 1.3
Failed to solve IK for trajectory point:  move #30 time: 1.4000000000000001
Failed to solve IK for trajectory point:  move #30 time: 1.5
Failed to solve IK for trajectory point:  move #30 time: 1.6
Failed to solve IK for trajectory point:  move #30 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #30 time: 1.8
Failed to solve IK for trajectory point:  move #30 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #30 turn (0.0)
Failed to solve IK for trajectory point:  place brick #30 turn (0.5)
Failed to solve IK for trajectory point:  place brick #30 turn (1.0)
Failed to solve IK for trajectory point:  place brick #30 turn (1.5)
Failed to solve IK for trajectory point:  place brick 

...

Failed to solve IK for trajectory point:  move #50 time: 1.0
Failed to solve IK for trajectory point:  move #50 time: 1.1
Failed to solve IK for trajectory point:  move #50 time: 1.2000000000000002
Failed to solve IK for trajectory point:  move #50 time: 1.3
Failed to solve IK for trajectory point:  move #50 time: 1.4000000000000001
Failed to solve IK for trajectory point:  move #50 time: 1.5
Failed to solve IK for trajectory point:  move #50 time: 1.6
Failed to solve IK for trajectory point:  move #50 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #50 time: 1.8
Failed to solve IK for trajectory point:  move #50 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #50 turn (0.0)
Failed to solve IK for trajectory point:  place brick #50 turn (0.5)
Failed to solve IK for trajectory point:  place brick #50 turn (1.0)
Failed to solve IK for trajectory point:  place brick #50 turn (1.5)
Failed to solve IK for trajectory point:  place brick 

Failed to solve IK for trajectory point:  move #58 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #58 approach (0.0)
Failed to solve IK for trajectory point:  place brick #58 approach (0.5)
Failed to solve IK for trajectory point:  place brick #58 approach (1.0)
Failed to solve IK for trajectory point:  place brick #58 approach (1.5)
Failed to solve IK for trajectory point:  place brick #58 approach (2.0)
Failed to solve IK for trajectory point:  place brick #58 approach (2.5)
Failed to solve IK for trajectory point:  place brick #58 approach (3.0)
Failed to solve IK for trajectory point:  place brick #58 approach (3.5)
Failed to solve IK for trajectory point:  place brick #58 finger open 1
Failed to solve IK for trajectory point:  place brick #58 finger open 2
Failed to solve IK for trajectory point:  place brick #58 withdraw (0.0)
Failed to solve IK for trajectory point:  place brick #58 withdraw (0.5)
Failed to solve IK for trajectory point:  place br

Failed to solve IK for trajectory point:  move #65 time: 1.3
Failed to solve IK for trajectory point:  move #65 time: 1.4000000000000001
Failed to solve IK for trajectory point:  move #65 time: 1.5
Failed to solve IK for trajectory point:  move #65 time: 1.6
Failed to solve IK for trajectory point:  move #65 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #65 time: 1.8
Failed to solve IK for trajectory point:  move #65 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #65 approach (0.0)
Failed to solve IK for trajectory point:  place brick #65 approach (0.5)
Failed to solve IK for trajectory point:  place brick #65 approach (1.0)
Failed to solve IK for trajectory point:  place brick #65 approach (1.5)
Failed to solve IK for trajectory point:  place brick #65 approach (2.0)
Failed to solve IK for trajectory point:  place brick #65 approach (2.5)
Failed to solve IK for trajectory point:  place brick #65 approach (3.0)
Failed to solve I

Failed to solve IK for trajectory point:  move #70 time: 1.0
Failed to solve IK for trajectory point:  move #70 time: 1.1
Failed to solve IK for trajectory point:  move #70 time: 1.2000000000000002
Failed to solve IK for trajectory point:  move #70 time: 1.3
Failed to solve IK for trajectory point:  move #70 time: 1.4000000000000001
Failed to solve IK for trajectory point:  move #70 time: 1.5
Failed to solve IK for trajectory point:  move #70 time: 1.6
Failed to solve IK for trajectory point:  move #70 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #70 time: 1.8
Failed to solve IK for trajectory point:  move #70 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #70 turn (0.0)
Failed to solve IK for trajectory point:  place brick #70 turn (0.5)
Failed to solve IK for trajectory point:  place brick #70 turn (1.0)
Failed to solve IK for trajectory point:  place brick #70 turn (1.5)
Failed to solve IK for trajectory point:  place brick 

Failed to solve IK for trajectory point:  move #78 time: 1.1
Failed to solve IK for trajectory point:  move #78 time: 1.2000000000000002
Failed to solve IK for trajectory point:  move #78 time: 1.3
Failed to solve IK for trajectory point:  move #78 time: 1.4000000000000001
Failed to solve IK for trajectory point:  move #78 time: 1.5
Failed to solve IK for trajectory point:  move #78 time: 1.6
Failed to solve IK for trajectory point:  move #78 time: 1.7000000000000002
Failed to solve IK for trajectory point:  move #78 time: 1.8
Failed to solve IK for trajectory point:  move #78 time: 1.9000000000000001
Failed to solve IK for trajectory point:  place brick #78 approach (0.0)
Failed to solve IK for trajectory point:  place brick #78 approach (0.5)
Failed to solve IK for trajectory point:  place brick #78 approach (1.0)
Failed to solve IK for trajectory point:  place brick #78 approach (1.5)
Failed to solve IK for trajectory point:  place brick #78 approach (2.0)
Failed to solve IK for tra

In [28]:
# Visualize coverage
vis = Visualizer(meshcat, kBrickGeom)
vis.clear_vis()
vis.visualize_coverage(planer.get_brick_poses()[covered_bricks], planer.get_brick_poses()[failed_bricks])

In [29]:
# Create robot
ilyich = IIWA_Ilyich(meshcat, kNumberOfBricks, kBrickGeom, [traj, finger_traj])

# Create default context
context = ilyich.CreateDefaultContext()

# Put all bricks in the warehouse
ilyich.put_bricks_in_warehouse(context, kWhLocation, kWhSize, kNumberOfBricks)

In [ ]:
#
# Move
#
# Move to calibration point and calibrate
sim, ctx = ilyich.work(None, context, sim_duration=trj_builder.get_trajectories().get_breakpoints()[0])
q0 = ilyich.get_q0(ctx)
print("calibration q0= ", q0)

# Continue for all bricks
for i in range(0, len(covered_bricks)):
    # Teleport the next new brick from the warehouse and unlock it
    ilyich.move_brick(context, X_WBrickSource, i)
    ilyich.unlock_brick(context, i)

    # Manipulate the brick to target
    sim, ctx = ilyich.work(sim, ctx, sim_duration=trj_builder.get_trajectories().get_breakpoints()[i+1])
    
    # Lock brick
    ilyich.lock_brick(context, i)